In [1]:
%load_ext autoreload
%autoreload 2

import os, yaml

import numpy as np

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
#from tensorflow.keras.callbacks import ModelCheckpoint

from encoders import SupervisedEncoder
from decoders import DecoderWithoutLatent, DecoderWithLatent
from models import SupervisedAutoencoder, UnsupervisedAutoencoder
from losses import SpectralLoss, MultiLoss
from preprocessing import F0LoudnessPreprocessor, MidiF0LoudnessPreprocessor
from dataloader import make_supervised_dataset
from callbacks import CustomWandbCallback, ModelCheckpoint

In [2]:
with open('configs/exp.yaml') as file:
    config = dict(yaml.load(file, Loader=yaml.FullLoader))
    data, train = config['data'], config['training']
    model_config, optim = config['model'], config['optimizer']

In [ ]:
train_set, validation_set, _ = make_supervised_dataset(data['path'],
                                                model_config['encoder'],
                                                train['batch_size'])

## Supervised DDSP

### TO DO

with logmag
without logmag

In [ ]:
preprocessing_timesteps = 250
decoder_timesteps = 1000

preprocessor = F0LoudnessPreprocessor(timesteps=preprocessing_timesteps)

# Without Latent
encoder = None
decoder = DecoderWithoutLatent(timesteps=decoder_timesteps)

# With Latent
#encoder = SupervisedEncoder()
#decoder = DecoderWithLatent(timesteps=decoder_timesteps)

# Choose a loss
loss = SpectralLoss(logmag_weight=1.0)
#loss = MultiLoss()

tracker_names = ['spec_loss'] if loss.name=='spectral_loss' else ['spec_loss', 'perc_loss', 'total_loss']

model = SupervisedAutoencoder(preprocessor=preprocessor,
                            encoder=encoder,
                            decoder=decoder,
                            loss_fn=loss,
                            tracker_names=tracker_names,
                            add_reverb=True)
                                
adam = Adam(learning_rate=ExponentialDecay(1e-3, decay_steps=10000, decay_rate=0.98))

model_dir = "model_checkpoints/yeah_{}".format(config['run_name'])
os.makedirs(model_dir, exist_ok=True)

#callbacks = [ModelCheckpoint(filepath=os.path.join(model_dir, 'model.ckpt'),
#                              monitor='val_spec_loss' if loss.name=='spectral_loss' else 'val_total_loss',
#                              save_best_only=True)]

callbacks = [ModelCheckpoint(model_dir,
               monitor='val_spec_loss' if loss.name=='spectral_loss' else 'val_total_loss')]

#csv_logger = tf.keras.callbacks.CSVLogger("logs/{}.csv".format(RUN_NAME), separator=",", append=False)
#callbacks = [ModelCheckpoint(model, RUN_NAME), csv_logger]
#callbacks.append(CustomWandbCallback(RUN_NAME)) # uncomment for WANDB

model.compile(adam)

In [ ]:
history = model.fit(train_set, validation_data=validation_set, epochs=10, callbacks=callbacks)

In [ ]:
preprocessing_timesteps = 250
decoder_timesteps = 1000

preprocessor = F0LoudnessPreprocessor(timesteps=preprocessing_timesteps)

# Without Latent
encoder = None
decoder = DecoderWithoutLatent(timesteps=decoder_timesteps)

# With Latent
#encoder = SupervisedEncoder()
#decoder = DecoderWithLatent(timesteps=decoder_timesteps)

# Choose a loss
loss = SpectralLoss(logmag_weight=1.0)
#loss = MultiLoss()

tracker_names = ['spec_loss'] if loss.name=='spectral_loss' else ['spec_loss', 'perc_loss', 'total_loss']

model = SupervisedAutoencoder(preprocessor=preprocessor,
                            encoder=encoder,
                            decoder=decoder,
                            loss_fn=loss,
                            tracker_names=tracker_names,
                            add_reverb=True)

## Unsupervised DDSP

In [ ]:
from metrics import f0_midi_scaled_L1_loss

In [ ]:
timesteps = 250

encoder_z = Encoder_z(timesteps=timesteps)
encoder_f0 = Encoder_f(timesteps=timesteps)

decoder = DecoderWithLatent()

preprocessor = MidiF0LoudnessPreprocessor(timesteps=timesteps)

#loss = SpectralLoss(logmag_weight=1.0)
loss = MultiLoss(logmag_weight=1.0, perceptual_loss_weight=38)

metric_fns = {"F0_recons_L1": f0_midi_scaled_L1_loss}

model = UnsupervisedAutoencoder(preprocessor=preprocessor,
                                encoder_f0=encoder_f0,
                                encoder_z=encoder_z,
                                decoder=decoder,
                                loss_fn=loss,
                                tracker_names=["total_loss", "spec_loss", "perc_loss","F0_recons_L1"],
                                metric_fns=metric_fns)

decay = ExponentialDecay(1e-3, decay_steps=10000, decay_rate=0.98)
adam = Adam(learning_rate=decay)

csv_logger = tf.keras.callbacks.CSVLogger("logs/{}.csv".format(RUN_NAME), separator=",", append=False)

callbacks = [ModelCheckpoint(model, RUN_NAME), csv_logger, CustomWandbCallback(RUN_NAME)]

model.compile(adam, metrics = [f0_midi_scaled_L1_loss])

In [ ]:
history = model.fit(train_set, 
                    validation_data=validation_set,
                    callbacks=callbacks, 
                    epochs=1000,
                    steps_per_epoch=train_set.my_len,
                    validation_steps=validation_set.my_len)

# TODO:

supervised data\
supervised model

interpolation/extrapolation (optional) \
test metrics


In [ ]:
#k_filters = [128]*2 + [256]*3 + [512]*4 + [1024]*3
#s_freqs = [1,1,2]*2 + [1,1,1,2] + [1,1]



In [ ]:
model.load("SupervisedViolinModel/300/model.ckpt")

In [ ]:
from dsp_utils.spectral_ops import compute_mfcc, compute_logmel, compute_loudness, compute_f0
def calculate_recons_f0_error(dataset):
    it = iter(dataset)
    preds,truth = [],[]
    for batch in it:
        pred = model(batch)
        preds.append(pred["audio_synth"].numpy())
        truth.append(pred["inputs"]["f0_hz"].numpy())
    pred_f0 = [compute_f0(p[0], 16000, 250, viterbi=True)[0] for p in preds]
    error = np.mean(np.abs(np.array(truth)[:,0,:,0]-np.array(pred_f0)))
    hz_to_midi = core.hz_to_midi
    F0_RANGE = spectral_ops.F0_RANGE
    return hz_to_midi(error)/F0_RANGE
calculate_recons_f0_error(train_set)

In [ ]:
import yaml

In [ ]:
x = {'model': {'input': 3, 'output': 4}, 'decoder': False}

In [ ]:
with open('y.yaml', 'w') as f:
    yaml.dump(config, f)

In [ ]:
with open('x.yaml') as file:
    config = dict(yaml.load(file, Loader=yaml.FullLoader))

In [ ]:
!pwd

In [ ]:
os.path.dirname('lol/audio_clips/Violin.txt')

In [ ]:
config = {'run_name': 'lol',
         'wandb': {}, #leave empty for not using 'project_name': 'supervised'
         'data': {'path': "audio_clips/Violin",
                 'sample_rate': 16000,
                 'preprocessing_time': 250,
                  'sample_rate' = 16000},
         'model': {'encoder': False, # use an encoder or not
                   'decoder_time': 1000,
                   'reverb': True,
                   'loss': 'spectral',
                   'path': 'here'},
         'optimizer': {'name': 'Adam', # just for reference
                        'lr': 1e-3,
                        'decay_steps': 10000,
                        'decay_rate': 0.98},
         'training': {'batch_size':32,
                     'epochs': 1000}
        }

In [ ]:
if config['wandb']:
    print('lol')

In [ ]:
config